In [ ]:
import multiprocessing 

import six
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

!pip install git+https://github.com/google/qkeras

# Tensorflow Colab file with some modifications

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from qkeras import *

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
#train_labels, test_labels = get_one_hot(train_labels, 10), get_one_hot(test_labels, 10)#

train_images = train_images.reshape(train_images.shape + (1,)).astype("float32")
test_images = test_images.reshape(test_images.shape + (1,)).astype("float32")

In [ ]:
def get_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape + (1,)).astype("float32")
    x_test = x_test.reshape(x_test.shape + (1,)).astype("float32")

    x_train /= 256.0
    x_test /= 256.0

    x_mean = np.mean(x_train, axis=0)

    x_train -= x_mean
    x_test -= x_mean

    nb_classes = np.max(y_train)+1
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)

    return (x_train, y_train), (x_test, y_test)

(train_images, train_labels), (test_images, test_labels) = get_data()

In [ ]:
np.mean(test_images)

In [ ]:
intBits = 1
precision = 16

from tensorflow.keras.optimizers import Adam

model = models.Sequential()
#model.add(layers.Conv2D(32, (3,3), activation='tanh'))
model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_tanh({}, {})'.format(precision, intBits),
        name='c1'))
model.add(layers.MaxPooling2D((2, 2)))

#model.add(layers.Conv2D(64, (3,3), activation='tanh'))
model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_tanh({}, {})'.format(precision, intBits),
        name='c2'))
model.add(layers.MaxPooling2D((2, 2)))

#model.add(layers.Conv2D(32, (3,3), activation='tanh'))
model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_tanh({}, {})'.format(precision, intBits),
        name='c3'))

model.add(layers.Flatten())

#model.add(layers.Dense(64, activation='tanh'))
#model.add(layers.Dense(10))
model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
model.add(QActivation('quantized_tanh({}, {})'.format(precision, intBits)))
model.add(QDense(10, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d2'))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=7, 
                    validation_data=(test_images[:5000], test_labels[:5000]))

In [ ]:
import qkeras
from qkeras import *
from qkeras.utils import *

In [ ]:
model_save_quantized_weights(model)
print ("Done")

In [ ]:
model.evaluate(test_images[5000:], test_labels[5000:])

In [ ]:
### Evaluation in a loop
intBits = 1
histories = []
mymodels = []

for precision in [8, 12, 16, 32]:
  for epoch in range(3, 13):
    model = models.Sequential()
    #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          activation='quantized_tanh({}, {})'.format(precision, intBits),
          name='c1'))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          activation='quantized_tanh({}, {})'.format(precision, intBits),
          name='c2'))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          activation='quantized_tanh({}, {})'.format(precision, intBits),
          name='c3'))

    model.add(layers.Flatten())
    #model.add(layers.Dense(64, activation='relu'))
    model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          name='d1'))
    model.add(QActivation('quantized_tanh({}, {})'.format(precision, intBits)))
    model.add(layers.Dense(10))

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    history = model.fit(train_images[:48000], train_labels[:48000], epochs= epoch, 
                      validation_data=(train_images[48000:60000], train_labels[48000:60000]), verbose=False)
  
    model_save_quantized_weights(model)
    print ("Done")

    model.evaluate(test_images, test_labels)

    mymodels.append(model)
    histories.append(history)

In [ ]:
mymodels[3].layers

# Now we go for CORDIC 

In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(arr, precision, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  #xarr = 1.2075*np.ones(shape=(len(arr), len(arr[0])))
  #yarr = np.zeros(shape = (len(arr), len(arr[0])))

  xarr = 1.2075*np.ones(shape=arr.shape)
  yarr = np.zeros(shape = arr.shape)

  global  lookup_arctanh

  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(arr)
    
    xchange = m*sigma*2**(-i)*yarr
    ychange = sigma*2**(-i)*xarr
    arrchange = sigma*lookup_arctanh[i-1]

    xarr -= xchange
    yarr += ychange
    arr -= arrchange
  return quantized_bits(precision, 1, alpha=1)(yarr / xarr)


In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(x, precision, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  

  global  lookup_arctanh
  current_vector = np.array([1.2075, 0])
  z = x

  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(z)
    x = current_vector[0]
    y = current_vector[1]
    xnew = x
    ynew = y
    xnew = xnew - m*sigma*2**(-i)*y
    ynew = ynew + sigma*2**(-i)*x
    z = z - sigma*lookup_arctanh[i-1]
    current_vector = [xnew, ynew]

  
  ex = current_vector[0] + current_vector[1]
  eminusx = current_vector[0] - current_vector[1]

  temp1 = quantized_bits(precision,1,alpha=1)(ex - eminusx)
  temp2 = quantized_bits(precision, 1, alpha=1)(ex + eminusx)

  return quantized_bits(precision,1, alpha=1)(temp1/temp2)

In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(x, precision, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  
  x = 2*x
  global  lookup_arctanh
  current_vector = np.array([1.2075, 0])
  z = x
  
  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(z)
    x = current_vector[0]
    y = current_vector[1]
    xnew = x
    ynew = y
    xnew = xnew - m*sigma*2**(-i)*y
    ynew = ynew + sigma*2**(-i)*x
    z = z - sigma*lookup_arctanh[i-1]
    current_vector = [xnew, ynew]

  
  ex = current_vector[0] + current_vector[1]
  eminusx = current_vector[0] - current_vector[1]

  temp1 = quantized_bits(precision,1,alpha=1)(ex - 1)
  temp2 = quantized_bits(precision, 1, alpha=1)(ex + 1)

  return quantized_bits(precision,1, alpha=1)(temp1/temp2)

In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(arr, precision, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  

  global  lookup_arctanh
  current_vector = np.array([1.2075, 0])
  arr *= 2

  xarr = 1.2075*np.ones(shape=(len(arr), len(arr[0])))
  yarr = np.zeros(shape=(len(arr), len(arr[0])))

  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(arr)

    xchange = m*sigma*2**(-i)*yarr
    ychange = sigma*2**(-i)*xarr
    arrchange = sigma*lookup_arctanh[i-1]

    xarr -= xchange
    yarr += ychange
    arr -= arrchange
    #x = current_vector[0]
    #y = current_vector[1]
    #xnew = x
    #ynew = y
    #xnew = xnew - m*sigma*2**(-i)*y
    #ynew = ynew + sigma*2**(-i)*x
    #z = z - sigma*lookup_arctanh[i-1]
    #current_vector = [xnew, ynew]

  return quantized_bits(precision, 1, alpha=1)(2/(1 + xarr - yarr) - 1)

  #eminus2x = xarr - yarr
  #temp1 = quantized_bits(9, 1, alpha=1)(1 + eminus2x)
  #temp2 = quantized_bits(9, 1, alpha=1)(1/temp1)
  #temp3 = quantized_bits(9, 1, alpha=1)(2*temp2)
  #return quantized_bits(9, 1, alpha=1)(temp3 - 1)

  #eminus2x = current_vector[0] - current_vector[1]
  #temp1 = quantized_bits(9, 1, alpha=1)(1 + eminus2x)
  #temp2 = quantized_bits(9, 1, alpha=1)(1/ temp1)
  #temp3 = quantized_bits(9, 1, alpha = 1)(2*temp2)
  
  #return quantized_bits(9,1, alpha=1)(temp3 - 1)


In [ ]:
qmodel = 0 


def epicGeneratePredictions(indices, precision, iterations):
  global qmodel
  x_test = test_images
  from keras import backend as K
  get_sixth_layer_output = K.function([qmodel.layers[0].input],
                                    [qmodel.layers[6].output])
 
  layer6_output = get_sixth_layer_output([x_test])[0]

  layer6_output = layer6_output[indices[0]: indices[1]]

  layer7_output = modifiedCordicTanh(layer6_output, precision, iterations)

  input_shape = qmodel.layers[8].get_input_shape_at(0)
  layer_input = Input(shape=(64))
  x = layer_input
  x = qmodel.layers[8](x)
  qm4 = Model(layer_input, x)

  predictions = np.array(qm4.predict(layer7_output))

  #predictions = modifiedCordicTanh(predictions, precision, iterations)

  a = predictions
  return (a == a.max(axis=1)[:,None]).astype(int)
"""

def epicGeneratePredictions(indices, precision, iterations):
  global qmodel
  x_test = test_images
  from keras import backend as K

  #This generates output of 1st Conv2D layer
  get_zeroth_layer_output = K.function([qmodel.layers[0].input],
                                    [qmodel.layers[0].output])
  
  layer0_output = get_zeroth_layer_output([x_test])[0]

  layer0_output = layer0_output[indices[0]: indices[1]]

  #print (layer0_output.shape)
  #This generates output of first QActivation
  layer1_output = modifiedCordicTanh(layer0_output, precision, iterations)

  # This generates output of 2nd Conv2D layer
  input_shape = qmodel.layers[2].get_input_shape_at(0)
  layer_input = Input(input_shape)
  x = layer_input
  x = qmodel.layers[3](qmodel.layers[2](x))
  qm4 = Model(layer_input, x)

  predictions = np.array(qm4.predict(layer1_output))
  del layer1_output
  del layer0_output
  del get_zeroth_layer_output
  #This generates output of 2nd QActivation layer
  predictions = modifiedCordicTanh(predictions, precision, iterations)

  #This generates output of the 3rd QConv2D layer
  input_shape = qmodel.layers[5].get_input_shape_at(0)
  layer_input = Input(input_shape)
  x = layer_input
  x = qmodel.layers[6](qmodel.layers[5](x))
  qm4 = Model(layer_input, x)

  predictions = np.array(qm4.predict(predictions))

  #This generates output of 3rd Qactivation layer
  predictions = modifiedCordicTanh(predictions, precision, iterations)

  #This generates output of 1st QDense layer
  input_shape = qmodel.layers[8].get_input_shape_at(0)
  layer_input = Input(input_shape)
  x = layer_input
  x = qmodel.layers[9](qmodel.layers[8](x))
  qm4 = Model(layer_input, x)

  predictions = np.array(qm4.predict(predictions))

  #This generates output of 1st Dense QActivation
  predictions = np.array(qm4.predict(predictions))

  #This generates the last layer's output
  input_shape = qmodel.layers[11].get_input_shape_at(0)
  layer_input = Input(input_shape)
  x = layer_input
  x = qmodel.layers[11](x)
  qm4 = Model(layer_input, x)

  predictions = np.array(qm4.predict(predictions))
  del q4
  del x
  del get_input_shape
  del layer_input

  a = predictions
  return (a == a.max(axis=1)[:,None]).astype(int)
  """

In [ ]:

def modelMaker(precision, intBits, epoch=10):
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_tanh({}, {})'.format(precision, intBits),
        name='c1'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_tanh({}, {})'.format(precision, intBits),
        name='c2'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_tanh({}, {})'.format(precision, intBits),
        name='c3'))

  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(QActivation('quantized_tanh({}, {})'.format(precision, intBits)))
  model.add(QDense(10, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d2'))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              #loss = "categorical_crossentropy",
              metrics=['accuracy'])

  history = model.fit(train_images[:48000], train_labels[:48000], epochs= epoch, 
                    validation_data=(train_images[48000:], train_labels[48000:]), verbose=False)
  
  model_save_quantized_weights(model)
  return model

"""
def modelMaker(precision, intBits):
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='c1'))
  model.add(QActivation('quantized_tanh({}, {})'.format(precision, intBits)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='c2'))
  model.add(QActivation('quantized_tanh({}, {})'.format(precision, intBits)))

  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='c3'))
  model.add(QActivation('quantized_tanh({}, {})'.format(precision, intBits)))

  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(QActivation('quantized_tanh({}, {})'.format(precision, intBits)))
  model.add(QDense(10, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d2'))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              #loss = "categorical_crossentropy",
              metrics=['accuracy'])

  history = model.fit(train_images[:48000], train_labels[:48000], epochs= 5 + int(3*(precision/4 - 2)), 
                    validation_data=(train_images[48000:], train_labels[48000:]), verbose=False)
  
  model_save_quantized_weights(model)
  return model
  """


## 8bit

In [ ]:
for epoch in range(3, 15):
  qmodel = modelMaker(9, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 9,iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
# ex - e-x / ex + e-x

for epoch in range(3, 15):
  qmodel = modelMaker(9, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 9,iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
# e2x - 1/ e2x + 1

for epoch in range(3, 15):
  qmodel = modelMaker(9, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 9,iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
#2sig2x - 1

for epoch in range(3, 15):
  qmodel = modelMaker(9, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 9,iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
qmodel = modelMaker(9, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([0,10000], 9, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[0:10000], 10)
#y_test = test_labels[0:10000]
for i in answerVectors:
  correct = 0
  for j in range(10000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

In [ ]:
qmodel.summary()

In [ ]:
answerVectors

In [ ]:
accuracy = []
y_test = get_one_hot(test_labels[0:10000], 10)
#y_test = test_labels[0:10000]
for i in answerVectors:
  correct = 0
  for j in range(10000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

In [ ]:
y_test

## 12 bit

In [ ]:
for epoch in range(3, 15):
  qmodel = modelMaker(13, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 13, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
qmodel = modelMaker(13, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([0, 10000], 13, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[0:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(10000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

## 16 bit

In [ ]:
for epoch in range(3, 15):
  qmodel = modelMaker(17, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 17, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
qmodel = modelMaker(17, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([0, 10000], 17, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[0:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(10000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

## 24 bit

In [ ]:
for epoch in range(3, 15):
  qmodel = modelMaker(25, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 25, iter))
    print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
for epoch in range(14, 20):
  qmodel = modelMaker(25, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 25, iter))
    print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

## 32 Bit

In [ ]:
qmodel = modelMaker(33, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([0, 10000], 33, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[0:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(10000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

# for 24 bit only

## Tensorflow

In [ ]:
### Evaluation in a loop
intBits = 1
histories = []
mymodels = []

for precision in [24]:
  for epoch in range(3, 15):
    model = models.Sequential()
    #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          activation='quantized_tanh({}, {})'.format(precision, intBits),
          name='c1'))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          activation='quantized_tanh({}, {})'.format(precision, intBits),
          name='c2'))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          activation='quantized_tanh({}, {})'.format(precision, intBits),
          name='c3'))

    model.add(layers.Flatten())
    #model.add(layers.Dense(64, activation='relu'))
    model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          name='d1'))
    model.add(QActivation('quantized_tanh({}, {})'.format(precision, intBits)))
    model.add(layers.Dense(10))

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    history = model.fit(train_images[:48000], train_labels[:48000], epochs= epoch, 
                      validation_data=(train_images[48000:60000], train_labels[48000:60000]), verbose=False)
  
    model_save_quantized_weights(model)
    print ("Done")

    model.evaluate(test_images, test_labels)

    mymodels.append(model)
    histories.append(history)